# BigTIFF

* [tiff format](http://www.fileformat.info/format/tiff/corion.htm)
* [bigtiff, awaresystems](https://www.awaresystems.be/imaging/tiff/bigtiff.html)
* [bigtiff, org](http://bigtiff.org/)
* [GDAL, frmts, gtiff](https://gdal.org/drivers/raster/gtiff.html)

In [1]:
!dir ../tests/data/BigTIFF

BigTIFFLong8.tif	       BigTIFFMotorola.tif  BigTIFF.tif
BigTIFFLong8Tiles.tif	       BigTIFFSamples.html  Classic-test.tif
BigTIFFLong.tif		       BigTIFFSubIFD4.tif   Classic.tif
BigTIFFMotorolaLongStrips.tif  BigTIFFSubIFD8.tif   test.tif


In [2]:
ctif = "../tests/data/BigTIFF/Classic.tif"
btif = "../tests/data/BigTIFF/BigTIFF.tif"

# Binary

In [3]:
import struct

with open(ctif, "rb") as cf, open(btif, "rb") as bf:
    ibyte = 2
    ioffset = 0

    # cf.seek(ibyte, ioffset)
    # ioffset = ioffset + ibyte
    # 
    # cbyte = cf.read(ibyte)

    i = 0
    cbyte = cf.read(ibyte)
    bbyte = bf.read(ibyte)
    cbyteorder = cbyte.decode('utf-8')
    bbyteorder = bbyte.decode('utf-8')
    print("%06d" % (i * ibyte),
          "\t", "ctif:", cbyteorder,
          "\t", "btif:", bbyteorder
         )
    
    if cbyte.decode('utf-8')=="II" and bbyte.decode('utf-8')=="II":
        byteorder = "little"
        signed = False
    
    i += 1
    # Do stuff with byte.
    cbyte = cf.read(ibyte)
    bbyte = bf.read(ibyte)
    print("%06d" % (i * ibyte),
          "\t", "ctif:", int.from_bytes(cbyte, byteorder=byteorder, signed=signed),
          "\t", "btif:", int.from_bytes(bbyte, byteorder=byteorder, signed=signed)
         )
    
    # while cbyte != b"" or bbyte != b"":
    for j in range(20):
        i += 1
        # Do stuff with byte.
        cbyte = cf.read(ibyte)
        bbyte = bf.read(ibyte)
        print("%06d" % (i * ibyte),
              "\t", "ctif:", int.from_bytes(cbyte, byteorder=byteorder, signed=signed),
              "\t", "btif:", int.from_bytes(bbyte, byteorder=byteorder, signed=signed)
             )

000000 	 ctif: II 	 btif: II
000002 	 ctif: 42 	 btif: 43
000004 	 ctif: 12302 	 btif: 8
000006 	 ctif: 0 	 btif: 0
000008 	 ctif: 255 	 btif: 12304
000010 	 ctif: 65280 	 btif: 0
000012 	 ctif: 0 	 btif: 0
000014 	 ctif: 255 	 btif: 0
000016 	 ctif: 65280 	 btif: 255
000018 	 ctif: 0 	 btif: 65280
000020 	 ctif: 255 	 btif: 0
000022 	 ctif: 65280 	 btif: 255
000024 	 ctif: 0 	 btif: 65280
000026 	 ctif: 255 	 btif: 0
000028 	 ctif: 65280 	 btif: 255
000030 	 ctif: 0 	 btif: 65280
000032 	 ctif: 255 	 btif: 0
000034 	 ctif: 65280 	 btif: 255
000036 	 ctif: 0 	 btif: 65280
000038 	 ctif: 255 	 btif: 0
000040 	 ctif: 65280 	 btif: 255
000042 	 ctif: 0 	 btif: 65280


# GDAL

In [4]:
!gdalinfo --version
!echo "====="
!gdalinfo -mm {btif}

GDAL 3.1.0dev-3993eb2c54e66ffb5158eaa1ffd77dec8b98cadd, released 2019/09/13
=====
Driver: GTiff/GeoTIFF
Files: ../tests/data/BigTIFF/BigTIFF.tif
Size is 64, 64
Image Structure Metadata:
  INTERLEAVE=PIXEL
Corner Coordinates:
Upper Left  (    0.0,    0.0)
Lower Left  (    0.0,   64.0)
Upper Right (   64.0,    0.0)
Lower Right (   64.0,   64.0)
Center      (   32.0,   32.0)
Band 1 Block=64x42 Type=Byte, ColorInterp=Red
    Computed Min/Max=0.000,255.000
Band 2 Block=64x42 Type=Byte, ColorInterp=Green
    Computed Min/Max=0.000,255.000
Band 3 Block=64x42 Type=Byte, ColorInterp=Blue
    Computed Min/Max=0.000,255.000


In [5]:
try:
    from osgeo import gdal, osr, gdalconst
except ImportError:
    import gdal
    import osr
    import gdalconst

    # this allows GDAL to throw Python Exceptions
gdal.UseExceptions()

#  get raster datasource
src_ds = gdal.Open(btif, gdalconst.GA_ReadOnly)
print(src_ds)

# Description lost
print(src_ds.GetDescription())

# Metadata
print(src_ds.GetMetadata())

# Metadata band
print(src_ds.GetRasterBand(1).GetMetadata())
print(src_ds.GetRasterBand(1).ReadAsArray())

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x7f30c430af60> >
../tests/data/BigTIFF/BigTIFF.tif
{}
{}
[[255 255 255 ...,   0   0   0]
 [255 255 255 ...,   0   0   0]
 [255 255 255 ...,   0   0   0]
 ..., 
 [  0   0   0 ...,   0   0   0]
 [  0   0   0 ...,   0   0   0]
 [  0   0   0 ...,   0   0   0]]


# rasterio

In [6]:
import rasterio
import rasterio.features
import rasterio.warp

with rasterio.open(btif) as dataset:
    print(dataset.name)
    print(dataset.count)
    print(dataset.bounds)
    print(dataset.transform)
    print(dataset.crs)
    
    print({i: dtype for i, dtype in zip(dataset.indexes, dataset.dtypes)})
    
    print(dataset.dataset_mask())
    print(dataset.read(1))

../tests/data/BigTIFF/BigTIFF.tif
3
BoundingBox(left=0.0, bottom=64.0, right=64.0, top=0.0)
| 1.00, 0.00, 0.00|
| 0.00, 1.00, 0.00|
| 0.00, 0.00, 1.00|
None
{1: 'uint8', 2: 'uint8', 3: 'uint8'}
[[255 255 255 ..., 255 255 255]
 [255 255 255 ..., 255 255 255]
 [255 255 255 ..., 255 255 255]
 ..., 
 [255 255 255 ..., 255 255 255]
 [255 255 255 ..., 255 255 255]
 [255 255 255 ..., 255 255 255]]
[[255 255 255 ...,   0   0   0]
 [255 255 255 ...,   0   0   0]
 [255 255 255 ...,   0   0   0]
 ..., 
 [  0   0   0 ...,   0   0   0]
 [  0   0   0 ...,   0   0   0]
 [  0   0   0 ...,   0   0   0]]


/usr/local/lib/python3.6/dist-packages/rasterio/__init__.py:216: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


# xarray

In [7]:
import xarray as xr

with xr.open_rasterio(btif) as dataset:
    print(dataset.attrs)
    print(dataset.dims)
    print(dataset.coords)
    
    print(dataset.values[0])

OrderedDict([('transform', (1.0, 0.0, 0.0, 0.0, 1.0, 0.0)), ('res', (1.0, -1.0)), ('is_tiled', 0), ('nodatavals', (nan, nan, nan)), ('scales', (1.0, 1.0, 1.0)), ('offsets', (0.0, 0.0, 0.0))])
('band', 'y', 'x')
Coordinates:
  * band     (band) int64 1 2 3
  * y        (y) float64 0.5 1.5 2.5 3.5 4.5 5.5 ... 59.5 60.5 61.5 62.5 63.5
  * x        (x) float64 0.5 1.5 2.5 3.5 4.5 5.5 ... 59.5 60.5 61.5 62.5 63.5
[[255 255 255 ...,   0   0   0]
 [255 255 255 ...,   0   0   0]
 [255 255 255 ...,   0   0   0]
 ..., 
 [  0   0   0 ...,   0   0   0]
 [  0   0   0 ...,   0   0   0]
 [  0   0   0 ...,   0   0   0]]


/usr/local/lib/python3.6/dist-packages/rasterio/__init__.py:216: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
